## Installing the libraries

In [ ]:
!pip install -U langchain-community
!pip install -U langchain
!pip install langchain
!pip install chromadb
!pip install nltk
!pip install memory_profiler
!pip install datasets
!pip install langchain_google_genai
!pip install faiss-gpu
!pip install pynytimes
!pip install pandas

  Using cached faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (85.5 MB)


### Installing all the libraries

In [ ]:
import os
import re

import nltk
import psutil
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from googlesearch import search
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

from langchain_text_splitters import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains.question_answering import load_qa_chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata

# Download NLTK punkt tokenizer data
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import userdata


# To use it make sure to add your Gemini api key in the colab secrets with `GOOGLE_API_KEY` as the variable name

google_api_key = userdata.get('GOOGLE_API_KEY')




GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY


HF_TOKEN = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = HF_TOKEN


# Load the language model
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest", temperature=0.5
)


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

genai.configure(api_key=google_api_key)


In [ ]:
data=pd.read_csv("/content/NEW_NYT_prompts_with_human_story.csv")

In [ ]:
data=data.dropna()

In [ ]:
data.info()

In [ ]:
data.to_csv("NEW_NYT_prompts_with_human_story_.csv")

### Making function to fetch article from NYT link

In [ ]:
from pynytimes import NYTAPI

api_key = "zJK0Mfsv3SUGBsIMlm9h0rXS6NEtDAAu"

nyt = NYTAPI(api_key, parse_dates=True)

In [ ]:
sections = nyt.section_list()

In [ ]:
sections = [s["section"] for s in sections]
sections

['admin',
 'arts',
 'automobiles',
 'books',
 'briefing',
 'business',
 'climate',
 'corrections',
 'education',
 'en español',
 'fashion',
 'food',
 'gameplay',
 'guide',
 'health',
 'home & garden',
 'home page',
 'job market',
 'the learning network',
 'lens',
 'magazine',
 'movies',
 'multimedia/photos',
 'new york',
 'obituaries',
 'opinion',
 'parenting',
 'podcasts',
 'reader center',
 'real estate',
 'smarter living',
 'science',
 'sports',
 'style',
 'sunday review',
 't brand',
 't magazine',
 'technology',
 'theater',
 'times insider',
 'today’s paper',
 'travel',
 'u.s.',
 'universal',
 'the upshot',
 'video',
 'the weekly',
 'well',
 'world',
 'your money']

In [ ]:
categories = sections

In [ ]:
def fetch_article_data(categories):
  """Fetches article URLs, titles, and abstracts from The New York Times for specified categories and stores them in a pandas DataFrame.

  Args:
    categories (list): A list of The New York Times section names to fetch articles from.

  Returns:
    pandas.DataFrame: A DataFrame containing the collected article data (if successful).
  """

  # Create an empty DataFrame to store the article data
  article_data = pd.DataFrame(columns=["section", "title", "abstract","url"])

  # Create a New York Times API client (replace with your API key)
  api_key = "zJK0Mfsv3SUGBsIMlm9h0rXS6NEtDAAu"

  nyt = NYTAPI(api_key, parse_dates=True)

  for category in categories:
    # Fetch top stories from the specified category
    top_stories = nyt.top_stories(section=category)

    for story in top_stories:
      # Extract and store URL, title, and abstract in the DataFrame
      article_data = article_data._append({
          "section": story["section"],
          "title": story["title"],
          "abstract": story["abstract"],
          "url": story["url"]
      }, ignore_index=True)

  for category in categories:
    # Fetch top stories from the specified category
    top_stories = nyt.latest_articles(source="all", section=category)

    for story in top_stories:
      # Extract and store URL, title, and abstract in the DataFrame
      article_data = article_data._append({
          "section": story["section"],
          "title": story["title"],
          "abstract": story["abstract"],
          "url": story["url"]
      }, ignore_index=True)


  return article_data

# Example usage (replace 'YOUR_NEW_YORK_TIMES_API_KEY' with your actual API key)
# categories = [
#     "business", "food", "health",
#     "politics", "science",
#     "technology", "world"
# ]

# categories = [
#     "arts", "automobiles", "books", "business", "fashion", "food", "health",
#     "home", "insider", "magazine", "movies", "national", "nyregion",
#     "obituaries", "opinion", "politics", "realestate", "science", "sports",
#     "sundayreview", "technology", "theater", "tmagazine", "travel",
#     "world"
# ]


df = fetch_article_data(categories)


In [ ]:
top_stories = nyt.latest_articles(source="all", section="admin")

In [ ]:
top_stories[0]

{'slug_name': 'vid-japan-randoseru-37218',
 'section': 'Admin',
 'subsection': '',
 'title': '',
 'abstract': '',
 'uri': 'nyt://video/17d37cfb-dffd-56ae-bf2d-23402c799ef3',
 'url': 'https://www.nytimes.com/video/embedded/admin/100000009540636/vid-japan-randoseru-37218.html',
 'byline': 'By Noriko Hayashi',
 'item_type': 'Video',
 'source': 'New York Times',
 'updated_date': datetime.datetime(2024, 6, 26, 1, 26, 2, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),
 'created_date': datetime.datetime(2024, 6, 26, 1, 26, 2, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),
 'published_date': datetime.datetime(2024, 6, 26, 1, 26, 2, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),
 'first_published_date': datetime.datetime(2024, 6, 26, 1, 26, 2, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),
 'material_type_facet': 'Video',
 'kicker': '',
 'subheadline': '',
 'des_facet': [],
 'org_facet': [],
 'per_fac

In [ ]:
articles = nyt.article_search(query="Obama")


In [ ]:
df.to_csv("JUst_testing.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   section   1313 non-null   object
 1   title     1313 non-null   object
 2   abstract  1313 non-null   object
 3   url       1313 non-null   object
dtypes: object(4)
memory usage: 41.2+ KB


In [ ]:
df.dropna()

,section,title,abstract,url
0,movies,Kevin Costner Is Pursuing His Western Dream. W...,"To make “Horizon,” he put his own money on the...",https://www.nytimes.com/2024/06/25/movies/kevi...
1,arts,Mavis Staples Is an American Institution. She’...,"After more than seven decades onstage, the gos...",https://www.nytimes.com/2024/06/25/arts/music/...
2,arts,Age and the Image of Capacity,President Biden’s toughest opponent may not be...,https://www.nytimes.com/2024/06/25/arts/televi...
3,arts,‘Doctor Who’ Season in Review: The Doctor Migh...,Ncuti Gatwa shined as the 15th Doctor. But the...,https://www.nytimes.com/2024/06/25/arts/televi...
4,arts,‘Babylon Berlin’ Review: Dancing While the Wor...,The long-awaited fourth season of the cult-fav...,https://www.nytimes.com/2024/06/25/arts/televi...
...,...,...,...,...
1308,World,,,https://www.nytimes.com/video/embedded/world/a...
1309,World,,,https://www.nytimes.com/video/embedded/world/a...
1310,World,,,https://www.nytimes.com/video/embedded/world/a...
1311,World,,,https://www.nytimes.com/video/embedded/world/a...


In [ ]:
def fetch_article_text_sequential(url):
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    exclude=["Thank you for your patience","Subscribe","subscribe","trouble retrieving the article content","browser settings",
    "Thank you for your patience while we verify access. If you are in Reader mode please exit and log into your Times account, or subscribe for all of The Times.",
    "Thank you for your patience while we verify access.",
    "Already a subscriber? Log in.",
    "Want all of The Times? Subscribe.",
    "Advertisement",
    "Site Index",
    "Thank you for your patience while we verify access. If you are in Reader mode please exit andlog intoyour Times account, orsubscribefor all of The Times.",
    "Already a subscriber?Log in.",
    "Want all of The Times?Subscribe.",
    "Site Information Navigation"
    ]

    try:

        # Send a request to the webpage with the specified headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check that the request was successful

        # Parse the webpage content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Initialize an empty list to store the text sequentially
        article_content = []

        # Define the tags we are interested in (headlines and paragraphs)
        tags_of_interest = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']

        # Find all tags of interest in the order they appear in the document
        for tag in soup.find_all(tags_of_interest):
          if not any(excluded_phrase in tag.get_text() for excluded_phrase in exclude):
            text = tag.get_text(strip=True)
            article_content.append(text)

        return '\n'.join(article_content)

    except:
      return None



# Example usage:
# url = "https://www.nytimes.com/2024/05/14/us/politics/primaries-maryland-west-virginia-nebraska.html?smtyp=cur"
url = "https://www.nytimes.com/2024/06/10/technology/personaltech/apple-google-ai.html"
article_text = fetch_article_text_sequential(url)
print(article_text)

ArtificialIntelligence
ArtificialIntelligence
Supported by
tech fix
Welcome to the Era of the A.I. Smartphone
Apple and Google are getting up close and personal with user data to craft memos, summarize documents and generate images.
ByBrian X. Chen
Every year, Apple and Google announce major software updates that bring new features to our smartphones, like cosmetic overhauls to the home screen, stronger privacy protections and fun messaging tools. This year, the changes will feel more radical because the companies are focusing on reinventing our phones with artificial intelligence.
At its annual software developer conference on Monday, Apple showed a host of enhancements coming this fall to iOS 18, its operating system powering iPhones. The new tools include arevamped version of its voice assistant, Siri, that is easier to talk to and an A.I. system that will generate images, create summaries of web articles and craft responses to text messages and emails.
Apple’s news followed Google’

In [ ]:
df['Human story fetched'] = df['url'].apply(fetch_article_text_sequential)


In [ ]:
df

,section,title,abstract,url,Human story fetched
0,movies,Kevin Costner Is Pursuing His Western Dream. W...,"To make “Horizon,” he put his own money on the...",https://www.nytimes.com/2024/06/25/movies/kevi...,What to Watch\nWhat to Watch\nSupported by\nKe...
1,arts,Mavis Staples Is an American Institution. She’...,"After more than seven decades onstage, the gos...",https://www.nytimes.com/2024/06/25/arts/music/...,Supported by\nMavis Staples Is an American Ins...
2,arts,Age and the Image of Capacity,President Biden’s toughest opponent may not be...,https://www.nytimes.com/2024/06/25/arts/televi...,critic’s notebook\nAge and the Image of Capaci...
3,arts,‘Doctor Who’ Season in Review: The Doctor Migh...,Ncuti Gatwa shined as the 15th Doctor. But the...,https://www.nytimes.com/2024/06/25/arts/televi...,What to Watch\nWhat to Watch\nSupported by\nCr...
4,arts,‘Babylon Berlin’ Review: Dancing While the Wor...,The long-awaited fourth season of the cult-fav...,https://www.nytimes.com/2024/06/25/arts/televi...,What to Watch\nWhat to Watch\nSupported by\n‘B...
...,...,...,...,...,...
1308,World,,,https://www.nytimes.com/video/embedded/world/a...,None
1309,World,,,https://www.nytimes.com/video/embedded/world/a...,None
1310,World,,,https://www.nytimes.com/video/embedded/world/a...,None
1311,World,,,https://www.nytimes.com/video/embedded/world/a...,None


In [ ]:
df.to_csv("new_testsforseq.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   section              1313 non-null   object
 1   title                1313 non-null   object
 2   abstract             1313 non-null   object
 3   url                  1313 non-null   object
 4   Human story fetched  559 non-null    object
dtypes: object(5)
memory usage: 51.4+ KB


In [ ]:
article_text

'ArtificialIntelligence\nArtificialIntelligence\nSupported by\ntech fix\nWelcome to the Era of the A.I. Smartphone\nApple and Google are getting up close and personal with user data to craft memos, summarize documents and generate images.\nByBrian X. Chen\nEvery year, Apple and Google announce major software updates that bring new features to our smartphones, like cosmetic overhauls to the home screen, stronger privacy protections and fun messaging tools. This year, the changes will feel more radical because the companies are focusing on reinventing our phones with artificial intelligence.\nAt its annual software developer conference on Monday, Apple showed a host of enhancements coming this fall to iOS 18, its operating system powering iPhones. The new tools include arevamped version of its voice assistant, Siri, that is easier to talk to and an A.I. system that will generate images, create summaries of web articles and craft responses to text messages and emails.\nApple’s news follow

### Making function for Zero shot prompting using a topic name

In [ ]:
def ai_generation_without_context(topic):
    prompt_template = PromptTemplate.from_template(
        "Generate a detailed, precise and ordered article in proper format for the NY times tweet on the topic:   {topic}"
    )
    prompt = prompt_template.format(topic=topic)


    res = model.invoke(prompt)

    res = res.content
    res = re.sub(r"[^a-zA-Z0-9\s]", "", res)
    print(res)


    return res

In [ ]:
response = ai_generation_without_context("At least eight people were killed and dozens were injured on Tuesday after a bus carrying farm workers collided with a pickup truck in Central Florida, local authorities said. About 40 passengers were taken to nearby hospitals to be treated for injuries")

 Eight Dead Dozens Injured in Florida Bus Crash Involving Farm Workers

LAKE WALES Fla  A devastating collision between a bus carrying farm workers and a pickup truck in Central Florida left eight people dead and dozens injured on Tuesday according to local authorities The accident occurred Insert specific location of the accident in Lake Wales sending shockwaves through the community and prompting a swift response from emergency services

Around Insert approximate time of the accident the bus carrying approximately Insert more precise number of passengers if available farm workers collided with the pickup truck The impact of the crash resulted in the immediate fatalities of eight individuals  

Emergency medical personnel arrived at the scene to find a chaotic and tragic situation  Approximately 40 passengers were transported to nearby hospitals for treatment with injuries ranging in severity Insert names of hospitals if available and appropriate and any information about the conditio

### Making Function for few shot prompting using human story as context
1. Through normal prompting
2. Utilizing vector store

In [ ]:
topic = "Can Apple Rescue the Vision Pro?"

In [ ]:
human_story = """"Supported by
The Shift
Can Apple Rescue the Vision Pro?
The $3,500 “spatial computing” device has gathered dust on my shelf. Can tweaks and upgrades save it from obsolescence?
ByKevin Roose
Reporting from Cupertino, Calif.
When I first got my hands on anApple Vision Proearly this year, it felt like magic.
I loved the $3,500 “spatial computing” headset, even though Icouldn’t really figure outwhat it was for. For weeks, I took it everywhere, enduring judgmental glares (or were they jealous stares?) from colleagues at the office, strangers at coffee shops and fellow passengers on airplanes. I even used the Vision Pro in the back seat of a self-driving Waymo car, which I believe qualifies me for some kind of “Mr. San Francisco” award.
But novelty fades, and today I barely use the Vision Pro at all. Every few weeks, I strap it to my head to do some focused writing, or watch a movie in bed while my wife sleeps. Otherwise, it sits on a shelf collecting dust.
Apple hasn’t released sales figures, but analysts’ estimatessuggestthat the device has been a flop, selling fewer units than expected. Social media isn’t buzzing with videos of enthusiastic“Vision Bros”wearing their headsets in public, as it was in the days after the device’s release. Some early adoptersreturned their Vision Prosfor refunds, and lightly used headsets aresellingfor as little as $2,500 on resale websites.
When I took a casual poll of other Vision Pro owners I know — mostly journalists and tech workers — I learned that few of them were using theirs, either.
“Haven’t touched mine in a month,” one friend texted. “It’s a shame, I was so bullish.”
At itsannual developer conferenceon Monday, Apple announced a few new features for the Vision Pro, including a new version of its VisionOS operating system, new gesture controls and a way of turning old photos into 3-D “spatial photos” that can be viewed on the device. Apple also said it would soon begin selling the Vision Pro in countries including China, Japan and Britain."
"""

In [ ]:
doc =  Document(page_content=human_story, metadata={"source": "NYT Articles"})


# Text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
chunking = text_splitter.split_documents([doc])



In [ ]:
doc =  Document(page_content=human_story, metadata={"source": "NYT articles"})


In [ ]:
doc.page_content

'"Supported by\nThe Shift\nCan Apple Rescue the Vision Pro?\nThe $3,500 “spatial computing” device has gathered dust on my shelf. Can tweaks and upgrades save it from obsolescence?\nByKevin Roose\nReporting from Cupertino, Calif.\nWhen I first got my hands on anApple Vision Proearly this year, it felt like magic.\nI loved the $3,500 “spatial computing” headset, even though Icouldn’t really figure outwhat it was for. For weeks, I took it everywhere, enduring judgmental glares (or were they jealous stares?) from colleagues at the office, strangers at coffee shops and fellow passengers on airplanes. I even used the Vision Pro in the back seat of a self-driving Waymo car, which I believe qualifies me for some kind of “Mr. San Francisco” award.\nBut novelty fades, and today I barely use the Vision Pro at all. Every few weeks, I strap it to my head to do some focused writing, or watch a movie in bed while my wife sleeps. Otherwise, it sits on a shelf collecting dust.\nApple hasn’t released s

In [ ]:
type(doc)

langchain_core.documents.base.Document

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)


In [ ]:
chunking = text_splitter.split_documents([doc])
chunking

[Document(page_content='"Supported by\nThe Shift\nCan Apple Rescue the Vision Pro?\nThe $3,500 “spatial computing” device has gathered dust on my shelf. Can tweaks and upgrades save it from obsolescence?\nByKevin Roose\nReporting from Cupertino, Calif.', metadata={'source': 'NYT articles'}),
 Document(page_content='ByKevin Roose\nReporting from Cupertino, Calif.\nWhen I first got my hands on anApple Vision Proearly this year, it felt like magic.', metadata={'source': 'NYT articles'}),
 Document(page_content='I loved the $3,500 “spatial computing” headset, even though Icouldn’t really figure outwhat it was for. For weeks, I took it everywhere, enduring judgmental glares (or were they jealous stares?) from colleagues at the office, strangers at coffee shops and', metadata={'source': 'NYT articles'}),
 Document(page_content='at the office, strangers at coffee shops and fellow passengers on airplanes. I even used the Vision Pro in the back seat of a self-driving Waymo car, which I believe 

In [ ]:

#Turn the chunks into embeddings and store them in Chroma
vectordb=Chroma.from_documents(chunking,embeddings)

#Configure Chroma as a retriever with top_k=5
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

#Create the retrieval chain
template = """
Generate a detailed precise and ordered article in proper format for the NY times tweet on the topic: `{input}`
context: {context}

answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":topic})

res = response["answer"]
cleaned_text = re.sub(r"[^a-zA-Z0-9\s]", "", res)

print(cleaned_text)
#Print the answer to the question
# print(response["answer"])

 Can Apple Rescue the Vision Pro

By Kevin Roose

Reporting from Cupertino Calif

When I first got my hands on an Apple Vision Pro early this year it felt like magic The device a highpowered computer strapped to my face promised to usher in a new era of spatial computing seamlessly blending the digital and physical worlds I spent hours marveling at its crisp displays intuitive hand tracking and immersive virtual environments

But novelty fades Today the 3500 Vision Pro gathers dust on a shelf a futuristic paperweight in my otherwise analog living room  

And Im not alone When I took a casual poll of other Vision Pro owners I know  mostly journalists and tech workers  I learned that few of them were using theirs either

Havent touched mine in a month one friend texted Its a shame I was so bullish

At its annual developer conference on Monday Apple announced a few new features for the Vision Pro including a new version of its VisionOS operating system new gesture controls and a way of tu

### Function through normal prompting

In [ ]:
human_story = """Global Economy
Global Economy
Supported by
European Union Hits E.V.s From China With Extra Tariffs Up to 38%
Leaders in Brussels, Washington and beyond are trying to curb China’s automobile ambitions amid rising trade tensions and fears of a glut of Chinese cars flooding global markets.
ByMelissa Eddy
Reporting from Berlin
The European Union said on Wednesday that it would impose additional tariffs of up to 38 percent on electric vehicles imported from China into the bloc, in what E.U. leaders called an effort to protect the region’s manufacturers from unfair competition.
The move, which comes a month after President Biden quadrupled U.S.tariffs on Chinese electric vehiclesto 100 percent, opens another front in escalating trade tensions with China amid growing fears about a glut of Chinese green tech goods flooding global markets.
The actions by the European Union and the United States also reflect the challenges that traditional automakers in Europe and the United States face from up-and-coming Chinese companies founded with a focus on electric vehicles and much lower cost bases than their rivals in the West.
But unlike U.S. carmakers, several of their European counterparts are deeply entwined in the Chinese market and their cars produced there will also be subject to the higher tariffs. They have criticized the European Union’s move to increase duties from 10 percent, fearing retaliation from China, as well as an increase in prices across the market and a drop in demand for battery-powered cars.
The increases announced on Wednesday, which come on top of the existing 10 percent duties, are preliminary and will take effect on July 4. They range from 17.4 percent to 38.1 percent for three of the leading Chinese manufacturers, including BYD, Geely and SAIC. The tariffs were calculated based on the level of cooperation with European officials, who have spent the past few months investigating the level of support from the Chinese government for these companies.
Other automakers producing electric vehicles in China, including European companies with factories or joint ventures there, face a tariff of 21 percent or 38.1 percent, the E.U. said. Those rates also depend on their cooperation with the investigation."""

In [ ]:
def ai_generation_with_context(topic,human_story):

    prompt_template = PromptTemplate.from_template(
        "Generate a detailed, precise and ordered article in proper format for the NY times tweet on the topic:  {topic}, context {human_story}"
    )
    prompt = prompt_template.format(topic=topic,human_story =human_story)


    res = model.invoke(prompt)
    res = res.content
    res = re.sub(r"[^a-zA-Z0-9\s]", "", res)

    print(res)


    return res

In [ ]:
response = ai_generation_with_context("Voters in Maryland, West Virginia and Nebraska go to the polls on Tuesday. Here is what to watch in today’s primaries`",human_story)

 Maryland West Virginia Nebraska Head to the Polls as Global Trade Tensions Simmer

Voters in Maryland West Virginia and Nebraska will cast their ballots in key primary elections today but the backdrop to these local contests is a global economy grappling with rising trade tensions The European Unions recent decision to impose tariffs on Chinese electric vehicles EVs highlights the complex challenges facing American voters and policymakers alike

While local issues like the economy crime and education will undoubtedly be top of mind for voters the global economic landscape is increasingly intertwined with domestic concerns The EUs move to slap tariffs of up to 38 on Chinese EVs mirrors similar actions taken by the Biden administration signaling a united front against what Western powers perceive as unfair Chinese trade practices 

This transatlantic alignment comes amidst fears of a Chinese EV surge flooding global markets potentially undercutting American and European manufacturers Bo

###Function utilizing vector store as a RAG model

In [ ]:
from langchain.prompts import PromptTemplate


In [ ]:
def ai_generation_with_context_and_vector_search(topic,human_story):


    doc =  Document(page_content=human_story, metadata={"source": "NYT Articles"})


    # Text splitting
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
    chunking = text_splitter.split_documents([doc])

    #Turn the chunks into embeddings and store them in Chroma
    vectordb=Chroma.from_documents(chunking,embeddings)

    #Configure Chroma as a retriever with top_k=5
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})

    #Create the retrieval chain
    template = """
    Generate a detailed, precise and ordered article in proper format for the NY times tweet on the topic: `{input}`
    context: {context}

    answer:
    """
    prompt = PromptTemplate.from_template(template)
    combine_docs_chain = create_stuff_documents_chain(model, prompt)
    retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

    #Invoke the retrieval chain
    response=retrieval_chain.invoke({"input":topic})

    #Print the answer to the question
    res = response["answer"]

    res = re.sub(r"[^a-zA-Z0-9\s]", "", res)


    print(res)

    cleaned_text = re.sub(r"[^a-zA-Z0-9\s]", "", res)

    return res

In [ ]:
topic = "European Union Hits E.V.s From China With Extra Tariffs Up to 38%"

In [ ]:
human_story = """Global Economy
Global Economy
Supported by
European Union Hits E.V.s From China With Extra Tariffs Up to 38%
Leaders in Brussels, Washington and beyond are trying to curb China’s automobile ambitions amid rising trade tensions and fears of a glut of Chinese cars flooding global markets.
ByMelissa Eddy
Reporting from Berlin
The European Union said on Wednesday that it would impose additional tariffs of up to 38 percent on electric vehicles imported from China into the bloc, in what E.U. leaders called an effort to protect the region’s manufacturers from unfair competition.
The move, which comes a month after President Biden quadrupled U.S.tariffs on Chinese electric vehiclesto 100 percent, opens another front in escalating trade tensions with China amid growing fears about a glut of Chinese green tech goods flooding global markets.
The actions by the European Union and the United States also reflect the challenges that traditional automakers in Europe and the United States face from up-and-coming Chinese companies founded with a focus on electric vehicles and much lower cost bases than their rivals in the West.
But unlike U.S. carmakers, several of their European counterparts are deeply entwined in the Chinese market and their cars produced there will also be subject to the higher tariffs. They have criticized the European Union’s move to increase duties from 10 percent, fearing retaliation from China, as well as an increase in prices across the market and a drop in demand for battery-powered cars.
The increases announced on Wednesday, which come on top of the existing 10 percent duties, are preliminary and will take effect on July 4. They range from 17.4 percent to 38.1 percent for three of the leading Chinese manufacturers, including BYD, Geely and SAIC. The tariffs were calculated based on the level of cooperation with European officials, who have spent the past few months investigating the level of support from the Chinese government for these companies.
Other automakers producing electric vehicles in China, including European companies with factories or joint ventures there, face a tariff of 21 percent or 38.1 percent, the E.U. said. Those rates also depend on their cooperation with the investigation."""

In [ ]:
response = ai_generation_with_context_and_vector_search(topic,human_story)

 Europe Slaps Tariffs on Chinese Electric Vehicles Escalating Trade Tensions

BRUSSELS  The European Union announced on Wednesday the imposition of significant tariffs on electric vehicles EVs imported from China a move likely to escalate trade tensions between the two economic powerhouses The tariffs ranging from 21 to 38 are intended to protect European automakers from what the bloc calls unfair competition

The decision which follows a monthslong investigation by European authorities centers on allegations of substantial government subsidies provided to Chinese EV manufacturers These subsidies the EU argues artificially lower the cost of Chinese EVs giving them an unfair advantage in the European market 

Among the companies facing the highest tariff rate of 381 are Chinese auto giants BYD Geely and SAIC The specific rates levied on each manufacturer were determined based on their level of cooperation with the EUs investigation into state support

European automakers producing EVs i

### Retrieving web content

> Add blockquote



In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import googlesearch
import urllib.parse
import time

In [ ]:

def fetch_sentences_from_html(html):
  try:
    # Parse the string with BeautifulSoup
    if html == None:
      return []
    soup = BeautifulSoup(html, 'html.parser')
    paragraphs = soup.find_all("p")
    text = " ".join(p.get_text() for p in paragraphs)
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return sentences
  except Exception as e:
    print(f"Failed to fetch {html}: {str(e)}")
    return []

def rank_sentences(sentences, query, top_n=5):
  if sentences == []:
    return []
  vectorizer = TfidfVectorizer().fit_transform([query] + sentences)
  vectors = vectorizer.toarray()
  query_vector = vectors[0]
  sentences_vectors = vectors[1:]
  cosine_similarities = cosine_similarity([query_vector], sentences_vectors).flatten()
  ranked_indices = cosine_similarities.argsort()[-top_n:][::-1]
  return [sentences[idx] for idx in ranked_indices]


def get_google_search_results(query, start=0):
    search_url = "https://www.google.com/search"
    params = {"q": query, "start": start}
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(search_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    search_results = []
    for g in soup.find_all(class_="g"):
        title = g.find("h3").text if g.find("h3") else "No title"
        link = g.find("a")["href"] if g.find("a") else "No link"

        if not link.lower().endswith(('.pdf', '.PDF')):
          search_results.append({"title": title, "link": link})

    return search_results

def filter_results_by_domain(results, domains):
    filtered_results = []
    for result in results:
        parsed_url = urllib.parse.urlparse(result["link"])
        domain = parsed_url.netloc
        if any(domain.endswith(d) for d in domains):
            filtered_results.append(result)
    return filtered_results

def get_top_sentences(url, num_sentences=5):
    try:

        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        paragraphs = soup.find_all("p")
        text = " ".join(p.get_text() for p in paragraphs)
        sentences = re.split(r'(?<=[.!?]) +', text)


        return sentences
    except Exception as e:
        return [f"Error retrieving content from {url}: {e}"]


domains = [
    "wikipedia.org", "nytimes.com", "cnn.com", "bbc.com", "theguardian.com",
    "forbes.com", "reuters.com", "cnbc.com", "bloomberg.com", "foxnews.com",
    "npr.org", "washingtonpost.com", "wsj.com", "aljazeera.com", "ft.com",
    "huffpost.com", "nationalgeographic.com", "scientificamerican.com",
    "nature.com", "time.com", "usatoday.com", "apnews.com", "abcnews.go.com",
    "cbsnews.com", "nbcnews.com", "news.yahoo.com", "theatlantic.com",
    "vox.com", "politico.com", "economist.com"
]

exclude=["Thank you for your patience","Subscribe","subscribe","trouble retrieving the article content","browser settings",
    "Thank you for your patience while we verify access. If you are in Reader mode please exit and log into your Times account, or subscribe for all of The Times.",
    "Thank you for your patience while we verify access.",
    "Already a subscriber? Log in.",
    "Want all of The Times? Subscribe.",
    "Advertisement",
    "Site Index",
    "Thank you for your patience while we verify access. If you are in Reader mode please exit andlog intoyour Times account, orsubscribefor all of The Times.",
    "Already a subscriber?Log in.",
    "Want all of The Times?Subscribe.",
    "Site Information Navigation",
    "Please enable JS and disable any ad blocker"
    ]



# Ask the user for a search query
# user_query = input("Enter a search query: ")

# t1=time.time()

def web_retrieval(user_query):


    print("----------------------------------------------")

    print(f"Searching for: {user_query}")

    print("----------------------------------------------")
    num_results_needed = 15  # Number of urls to retrieve

    all_results = []
    start = 0
    while len(all_results) < num_results_needed:
        results = get_google_search_results(user_query, start=start)
        filtered_results = filter_results_by_domain(results, domains)
        if len(filtered_results) == 0:
            filtered_results = results  # If no filtered results, use original results

        all_results.extend(filtered_results)
        all_results = all_results[:num_results_needed]  # Ensure no more than needed results
        start += 10

    all_sentences2 = []
    # Print the search results and top sentences from each URL
    for result in all_results:
        print(f"Title: {result['title']}")
        print(f"Link: {result['link']}")
        sentences = get_top_sentences(result['link'])
        top_sentences = rank_sentences(sentences, user_query)


        # all_sentences2.extend(top_sentences)
        for sentence in top_sentences:
            if not any(excluded_phrase in sentence for excluded_phrase in exclude):
              print(sentence)
              all_sentences2.append(sentence)
        print()
    # all_sentences2.extend(top_sentences)
    content_new = "\n".join(cont for cont in all_sentences2)

    print("-------------------CONTENT---------------------------")
    print(content_new)
    print("----------------------------------------------")
    # t2=time.time()
    # print(t2-t1)

    return content_new



In [ ]:


def get_google_search_results(query, start=0):
    search_url = "https://www.google.com/search"
    params = {"q": query, "start": start}
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(search_url, params=params, headers=headers)
    # response = requests.get(search_url, params=params, headers = {'User-agent': 'your bot 0.1'})
    soup = BeautifulSoup(response.text, "html.parser")

    search_results = []
    print(soup)
    for g in soup.find_all(class_="g"):
        title = g.find("h3").text if g.find("h3") else "No title"
        link = g.find("a")["href"] if g.find("a") else "No link"

        if not link.lower().endswith(('.pdf', '.PDF')):
          search_results.append({"title": title, "link": link})

    return search_results


In [ ]:
from googlesearch import search


In [ ]:
from googlesearch import search

# Function to perform Google search and get relevant URLs
def perform_google_search(query, num_urls):
    try:
        search_results = [j for j in search(query, num=15, stop=10, pause=2)]
        # Increase the number to get more results
        non_pdf_urls = [url for url in search_results if not url.lower().endswith(('.pdf', '.PDF'))]
        return non_pdf_urls[:num_urls]  # Return the specified number of URLs
    except Exception as e:
        print(f"Error performing Google search: {e}")
        return []

In [ ]:
perform_google_search(query,1)

Error performing Google search: HTTP Error 429: Too Many Requests


[]

In [ ]:
import requests
import time
from bs4 import BeautifulSoup

def get_google_search_results(query, start=0):
    search_url = "https://www.google.com/search"
    params = {"q": query, "start": start}
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = None
    for attempt in range(5):  # Retry up to 5 times
        try:
            response = requests.get(search_url, params=params, headers=headers)
            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 10))  # Use 'Retry-After' header if available
                print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
                time.sleep(retry_after)
            else:
                response.raise_for_status()  # Raise other HTTP errors
                break
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}. Retrying...")
            time.sleep(2 ** attempt)  # Exponential backoff

    if response is None:
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    search_results = []

    print(soup)

    for g in soup.find_all(class_="g"):
        title = g.find("h3").text if g.find("h3") else "No title"
        link = g.find("a")["href"] if g.find("a") else "No link"

        if not link.lower().endswith(('.pdf', '.PDF')):
            search_results.append({"title": title, "link": link})
        print(link)

    return search_results

# Example usage
query = "OpenAI GPT-4"
results = get_google_search_results(query)
for result in results:
    print(result)


Rate limit exceeded. Retrying after 10 seconds...
Rate limit exceeded. Retrying after 10 seconds...
Rate limit exceeded. Retrying after 10 seconds...
Rate limit exceeded. Retrying after 10 seconds...
Rate limit exceeded. Retrying after 10 seconds...
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><meta content="initial-scale=1" name="viewport"/><title>https://www.google.com/search?q=OpenAI+GPT-4&amp;start=0</title></head>
<body onload="e=document.getElementById('captcha');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}" style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px; overscroll-behavior:contain;">
<div style="max-width:400px;">
<hr noshade="" size="1" style="color:#ccc; background-color:#ccc;"/><br/>
<form action="index" id="captcha-form" method="post">
<noscript>
<div style="font-size:13px;">
  In order to con

In [ ]:
get_google_search_results(query)

[]

In [ ]:
query = 'Grab Them. Then Stump Them. ; Word puzzles on LinkedIn. Logic challenges in The Washington Post. For news publishers and tech sites looking to both entice and engage users, games are serious business.'

In [ ]:
content = web_retrieval(query)

----------------------------------------------
Searching for: Grab Them. Then Stump Them. ; Word puzzles on LinkedIn. Logic challenges in The Washington Post. For news publishers and tech sites looking to both entice and engage users, games are serious business.
----------------------------------------------


KeyboardInterrupt: 

In [ ]:
content

In [ ]:
query = 'Buyers Snap Up Aging and Empty Office Buildings for Deep Discounts'

In [ ]:
content = web_retrieval(query)

----------------------------------------------
Searching for: Buyers Snap Up Aging and Empty Office Buildings for Deep Discounts
----------------------------------------------


KeyboardInterrupt: 

In [ ]:
query = 'Losing a Bridge Upends Life in Baltimore (and It May Soon Get Worse)'

In [ ]:
content = web_retrieval(query)

----------------------------------------------
Searching for: Losing a Bridge Upends Life in Baltimore (and It May Soon Get Worse)
----------------------------------------------


### Getting the csv data from Google Sheets

In [ ]:
import re

def convert_google_sheet_url(url):
    # Regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'

    # Replace function to construct the new URL for CSV export
    # If gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'

    # Replace using regex
    new_url = re.sub(pattern, replacement, url)

    return new_url

In [ ]:
import pandas as pd

# Replace with your modified URL
url = "https://docs.google.com/spreadsheets/d/1V_3r5-AUzSp1GTpassot4duJtcSgPcwIPm_gKtrYK7g/edit?gid=1440407290#gid=1440407290"

new_url = convert_google_sheet_url(url)

print(new_url)
# https://docs.google.com/spreadsheets/d/1mSEJtzy5L0nuIMRlY9rYdC5s899Ptu2gdMJcIalr5pg/export?gid=1606352415&format=csv

https://docs.google.com/spreadsheets/d/1V_3r5-AUzSp1GTpassot4duJtcSgPcwIPm_gKtrYK7g/export?format=csv


In [ ]:
df = pd.read_csv(new_url)
df

,section,title,abstract,url,Human story fetched,AI generated article with context(Human Story),AI generated story(without context)
0,business,"Inflation slowed, good news as the Fed meets.",Federal Reserve officials are poised to announ...,https://www.nytimes.com/live/2024/06/12/busine...,U.S. Economy\nU.S. Economy\nSupported by\nInfl...,Inflation Cools in May Offering Hope as Fed G...,Inflation Cools Offering Hope as Fed Gathers\...
1,business,Buyers Snap Up Aging and Empty Office Building...,Bargain hunters are getting deals of up to 70 ...,https://www.nytimes.com/2024/06/12/business/di...,Supported by\nBuyers Snap Up Aging and Empty O...,Bargain Hunters Circle as Office Buildings Se...,Empty Towers Full Potential Investors See Opp...
2,business,What to Watch as the Fed Meets,Federal Reserve officials are expected to leav...,https://www.nytimes.com/2024/06/12/business/ec...,U.S. Economy\nU.S. Economy\nSupported by\nThe ...,The Fed Holds Steady One Rate Cut and a Watch...,All Eyes on Powell What to Watch as the Fed M...
3,business,Losing a Bridge Upends Life in Baltimore (and ...,Congestion has increased on other routes in th...,https://www.nytimes.com/2024/06/12/business/ke...,BaltimoreBridge Collapse\nLosing a Bridge Upen...,Losing a Bridge Upends Life in Baltimore and ...,Losing a Bridge Upends Life in Baltimore and ...
4,business,Paramount’s Merger Talks With Skydance Fall Apart,There were several hitches in the last week as...,https://www.nytimes.com/2024/06/11/business/me...,Supported by\nParamount’s Merger Talks With Sk...,Skydance and Paramount Merger Talks Collapse ...,Paramounts Ambitions Grounded Merger Talks Wi...
...,...,...,...,...,...,...,...
934,Sunday Review,‘Sesame Street’ Is Opening Up to Syrian Refugees,What happens when the people who invented educ...,https://www.nytimes.com/2020/01/31/sunday-revi...,Supported by\nnews analysis\n‘Sesame Street’ I...,Sesame Street Opens its Doors and Hearts to S...,A Familiar Friend on a Difficult Street Sesam...
935,Sunday Review,American Jews and Israeli Jews Are Headed for ...,Is the world ready for another Great Schism?,https://www.nytimes.com/2019/01/04/sunday-revi...,Supported by\nnews analysis\nAmerican Jews and...,A Recipe for Disaster Are American and Israel...,A Looming Schism American and Israeli Jews Gr...
936,Travel,A Look at Saudi Arabia’s Big Bet on Tourism,To witness the kingdom’s profound transformati...,https://www.nytimes.com/video/travel/100000009...,new video loaded:A Look at Saudi Arabia’s Big ...,Saudi Arabias Sands of Change A Gamble on Tou...,Sand and Skylines A Look at Saudi Arabias Big...
937,Video,‘His Name Was Bélizaire’: Rare Portrait of Ens...,The Met recently acquired “Bélizaire and the F...,https://www.nytimes.com/interactive/2023/08/14...,Comments\n‘His Name Was Bélizaire’: Rare Portr...,His Name Was Blizaire A Child Emerges from Hi...,His Name Was Blizaire Rare Portrait of Enslav...


In [ ]:
df['AI generated story'] = df['title'].apply(ai_generation_without_context)


 Inflation Cools Offering Hope as Fed Gathers 

By Your Name New York Times

City State  In a welcome development for policymakers and consumers alike inflation cooled in Month according to Source of Inflation Data eg the Labor Departments Consumer Price Index offering a glimmer of hope as the Federal Reserve convenes for its highly anticipated policy meeting 

The Specific Index Used eg CPI rose Percentage increase in the year through Month a notable slowdown from the Previous months percentage increase increase in the year through Previous month This marks the Ordinal number eg third consecutive month of declining inflation signaling a potential turning point in the fight against soaring prices

Key Drivers of the Slowdown

 Specific Sector 1 eg Energy prices Explain the sectors contribution to the slowdown including specific data points and reasons for the change
 Specific Sector 2 eg Used car prices Explain the sectors contribution to the slowdown including specific data points and

 Losing a Bridge Upends Life in Baltimore and It May Soon Get Worse

By Your Name

Baltimore MD   The closure of the EastWest artery known as the Harford Road Bridge has plunged this Baltimore community into a logistical and emotional tailspin offering a stark preview of what may become a more frequent reality as Americas infrastructure ages 

For residents of Hamilton and the surrounding neighborhoods the bridge was more than just a thoroughfare It was a lifeline connecting families to jobs schools and essential services Now what was once a quick trip across the Herring Run has become a tedious detour adding precious minutes even hours to commutes and errands 

Its been chaos says Sarah Jenkins a single mother of two who relies on the bridge to get to her job at Johns Hopkins Hospital My commute has doubled Im constantly late and the stress is unbearable

The bridges sudden closure in February due to safety concerns sent shockwaves through the community While city officials scramble t

 Paramounts Ambitions Grounded Merger Talks With Skydance Collapse

Los Angeles California  In a blow to Hollywood consolidation hopes Paramount Global has abruptly ended merger discussions with David Ellisons production company Skydance Media according to sources familiar with the matter The potential deal which had been under negotiation for weeks would have seen Paramount absorb the Top Gun Maverick producer bolstering its content library and production capabilities

While the exact reasons behind the breakdown remain undisclosed insiders point to disagreements over price and structure as likely culprits Paramount grappling with a turbulent media landscape and the ongoing writers strike was reportedly hesitant to meet Skydances valuation expectations 

The collapse marks a significant setback for both parties For Paramount it represents a missed opportunity to acquire a prolific studio with a strong track record particularly in the action and science fiction genres Skydance meanwhil

 Streaming Dreams Deferred Why the Paramount Deal Talks Fizzled 

By Your Name The New York Times

Published Date

For months whispers of a potential media megamerger captivated Hollywood Paramount Global the entertainment giant behind CBS Paramount Pictures and Pluto TV was reportedly in talks to be acquired by Comcast the telecommunications behemoth controlling NBCUniversal and Peacock This week however those discussions abruptly ended leaving industry insiders and analysts dissecting the reasons behind the breakdown 

While neither company has officially commented on the collapse of the deal several factors likely contributed to the decision

1 A Price Tag Too High Sources close to the negotiations suggest that a key sticking point was the valuation of Paramount Comcast already grappling with a debt load exceeding 100 billion was reportedly hesitant to meet the asking price set by Shari Redstone chairwoman of Paramounts controlling shareholder National Amusements 

2 Regulatory Road

 Europe Slaps Tariffs on Chinese Electric Vehicles Escalating Trade Tensions

BRUSSELS  The European Union announced on Tuesday the imposition of tariffs ranging up to 38 on imports of electric vehicles EVs from China escalating trade tensions between the two economic powerhouses The move which follows a ninemonth investigation is aimed at countering what the EU claims are unfair subsidies given to Chinese EV manufacturers

The tariffs which will remain in place for at least five years target Chinesemade electric cars and vans The European Commission the EUs executive arm concluded that Chinese companies benefit from significant state support including cheap raw materials lowinterest loans and tax breaks This the Commission argues gives Chinese manufacturers an unfair advantage and threatens the viability of the European EV industry

We have found clear evidence of dumping said Valdis Dombrovskis the EUs trade chief The dumping margins found are significant ranging from 202 to 248 This

...

 France in Shock as Conservative Leader Embraces Far Right 

PARIS  A seismic shift has struck the French political landscape as Name of Conservative Leader head of the Name of Conservative Party announced a strategic alliance with the farright Name of FarRight Party led by Name of FarRight Leader The move met with widespread shock and condemnation signals a potential normalization of extremist views within mainstream politics and raises concerns about the future of Frances political direction 

The alliance forged in the wake of the recent Mention triggering event eg election results political crisis will see the two parties collaborate on key policy areas such as List specific policy areas eg immigration national security economic reforms This unprecedented partnership grants the farright a level of influence and legitimacy previously unimaginable marking a significant departure from Frances traditional political landscape

Reactions to the announcement have been swift and severe Opp


 Biden to Push for Seized Russian Assets to Fund Ukraines Reconstruction at G7 Summit

By Your Name The New York Times

City State  President Biden will use the upcoming Group of Seven summit in Japan to rally allies behind a controversial proposal using billions of dollars in frozen Russian assets to help fund Ukraines longterm reconstruction 

The move which faces significant legal and diplomatic hurdles reflects the growing pressure on Western nations to find ways to support Ukraines recovery estimated to cost hundreds of billions of dollars  

Since the start of the war the United States and its allies have frozen over 300 billion in assets belonging to the Russian central bank  The Biden administration argues that these funds along with assets seized from sanctioned Russian oligarchs should be redirected to help rebuild Ukraines shattered infrastructure and economy 

However the proposal faces stiff opposition with some European countries expressing concerns about the legal prece

 US Bolsters Ukraines Defenses With Additional Patriot Missile Battery

By Your Name The New York Times

WASHINGTON  In a move signaling continued commitment to Ukraines defense against Russian aggression the Biden administration announced on Date its decision to send an additional Patriot missile battery to the wartorn nation This latest military aid package details of which are expected to be released in the coming days underscores the ongoing need to bolster Ukraines air defense capabilities against relentless Russian missile and drone attacks

The Patriot system renowned for its advanced radar and longrange interception capabilities has proven instrumental in defending Ukrainian cities and critical infrastructure from aerial bombardment The first USsupplied Patriot battery delivered in April has already demonstrated its effectiveness in intercepting Russian missiles contributing significantly to Ukraines resilience in the face of ongoing aggression

This latest deployment marks the

 Nuclear Power Is Hard A ClimateMinded Billionaire Wants to Make It Easier

By Your Name

Date

Nuclear power a controversial yet potent tool in the fight against climate change faces significant hurdles high costs complex regulations and public safety concerns Now a climateconscious billionaire is stepping into the fray aiming to simplify and accelerate the deployment of this carbonfree energy source

Billionaires name known for his investments in mention previous ventures preferably related to technology or energy has launched a new initiative aimed at streamlining the development and construction of nuclear power plants The endeavor dubbed Name of the initiative seeks to tackle the industrys longstanding challenges through a multipronged approach

1 Standardized Design Recognizing that each nuclear power plant currently requires a unique blueprint leading to cost overruns and delays Billionaires name advocates for a standardized reactor design This approach similar to the successful

 UN Cites Potential War Crimes by Both Sides in Aftermath of Hostage Rescue Fueling International Outrage 

By Your Name The New York Times

City State  In the wake of a daring hostage rescue operation that saw Israeli commandos storm a Gaza hospital basement the United Nations has issued a stark warning stating that actions taken by both Israel and Hamas during the ongoing conflict may constitute war crimes The statement delivered by the UN High Commissioner for Human Rights Volker Trk has ignited a firestorm of international condemnation and fueled anxieties over the escalating humanitarian crisis in Gaza 

Trk addressing the UN Human Rights Council in Geneva expressed grave concern over the mounting civilian casualties and the dire humanitarian situation in Gaza He explicitly condemned the hostagetaking and killings perpetrated by Hamas labeling them abhorrent acts that could amount to war crimes 

However Trk also directed sharp criticism towards Israel highlighting the devastating

In [ ]:
df

,section,title,abstract,url,Human story fetched,AI generated story
0,business,"Inflation slowed, good news as the Fed meets.",Federal Reserve officials are poised to announ...,https://www.nytimes.com/live/2024/06/12/busine...,U.S. Economy\nLive Updates: Inflation Cools Sl...,Inflation Cools Offering Hope as Fed Gathers ...
1,business,Buyers Snap Up Aging and Empty Office Building...,Bargain hunters are getting deals of up to 70 ...,https://www.nytimes.com/2024/06/12/business/di...,Supported by\nBuyers Snap Up Aging and Empty O...,Ghost Towers No More Buyers See Opportunity i...
2,business,What to Watch as the Fed Meets,Federal Reserve officials are expected to leav...,https://www.nytimes.com/2024/06/12/business/ec...,U.S. Economy\nWhat to Watch as the Fed Meets\n...,All Eyes on Interest Rates What to Watch as t...
3,business,Losing a Bridge Upends Life in Baltimore (and ...,Congestion has increased on other routes in th...,https://www.nytimes.com/2024/06/12/business/ke...,BaltimoreBridge Collapse\nLosing a Bridge Upen...,Losing a Bridge Upends Life in Baltimore and ...
4,business,Paramount’s Merger Talks With Skydance Fall Apart,There were several hitches in the last week as...,https://www.nytimes.com/2024/06/11/business/me...,Supported by\nParamount’s Merger Talks With Sk...,Paramounts Ambitions Grounded Merger Talks Wi...
...,...,...,...,...,...,...
207,climate,Nuclear Power Is Hard. A Climate-Minded Billio...,Work is starting in Wyoming coal country on a ...,https://www.nytimes.com/2024/06/11/climate/bil...,Climate Change\nClimate Change\nSupported by\n...,Nuclear Power Is Hard A ClimateMinded Billion...
208,world,"After the hostage rescue, the U.N. says action...",Israel’s Parliament revived legislation that w...,https://www.nytimes.com/2024/06/11/world/middl...,Supported by\nKey Developments\nAfter the host...,UN Cites Potential War Crimes by Both Sides i...
209,briefing,Tuesday Briefing,The U.N. passes a cease-fire resolution for Gaza.,https://www.nytimes.com/2024/06/11/briefing/is...,Supported by\nTuesday Briefing\nThe U.N. passe...,The Tuesday Briefing Insert Concise Attention...
210,world,Nigeria Confronts Its Worst Economic Crisis in...,People in Africa’s most populous nation are su...,https://www.nytimes.com/2024/06/11/world/afric...,Global Economy\nGlobal Economy\nSupported by\n...,Nigeria Confronts Its Worst Economic Crisis i...


In [ ]:
# Generate AI-generated articles with context for each topic and human story
df['AI generated article with context'] = df.apply(lambda row: ai_generation_with_context_and_vector_search(row['title'], row['Human story fetched']), axis=1)



In [ ]:
df[["Human story fetched",	"AI generated story",	"AI generated article with context"]]

In [ ]:
# Function to create the query and retrieve content
def create_web_retrieved_content(row):
    query = f"{row['title']} ; {row['abstract']}"
    web_content = web_retrieval(query)
    return web_content



In [ ]:
# Apply the function to each row to create a new column
df['web retrieved content'] = df.apply(create_web_retrieved_content, axis=1)


----------------------------------------------
Searching for: Inflation slowed, good news as the Fed meets. ; Federal Reserve officials are poised to announce their rate decision and economic projections mere hours after the latest inflation release.
----------------------------------------------
Title: The Fed expects to cut rates just once this year
Link: https://www.cnn.com/business/live-news/markets-fed-meeting-06-12-24/index.html
The S&P 500 gained 1.1% and the Nasdaq Composite rose 1.7%.
     
            Wall Street is awaiting the Federal Reserve’s June interest rate decision and latest set of rate projections due at 2 pm ET.
     
            The Federal Reserve is expected to keep interest rates at a 23-year high for the seventh consecutive meeting on Wednesday and signal that it will cut rates this year fewer times than previously thought.
     
            Investors and other market observers will be paying close attention to Fed officials’ latest economic forecasts — known

KeyboardInterrupt: 

### KL divergence Code

In [ ]:
import numpy as np
from collections import Counter
import math

def text_to_distribution(text):
    # Convert text to lowercase and remove non-alphabetic characters
    text = ''.join(c.lower() for c in text if c.isalpha() or c.isspace())
    # Count the occurrences of each character
    char_count = Counter(text)
    # Calculate total number of characters
    total_chars = sum(char_count.values())
    # Calculate probability distribution
    distribution = {char: count / total_chars for char, count in char_count.items()}
    return distribution

def kl_divergence(p, q):
    epsilon = 1e-10  # small epsilon value to avoid division by zero
    return sum(p[i] * math.log((p[i] + epsilon) / (q[i] + epsilon)) for i in p.keys())

def calculate_kl_divergence(text1, text2):
    # Convert texts to probability distributions
    dist1 = text_to_distribution(text1)
    dist2 = text_to_distribution(text2)
    # Union of keys
    all_keys = set(dist1.keys()).union(set(dist2.keys()))

    # Fill missing keys with probability 0
    for key in all_keys:
        if key not in dist1:
            dist1[key] = 0
        if key not in dist2:
            dist2[key] = 0

    # Calculate KL divergence
    print(dist1)
    kl_div1 = kl_divergence(dist1, dist2)
    kl_div2 = kl_divergence(dist2, dist1)

    # return kl_div1, kl_div2
    return kl_div2






# Example usage
text1 = "This is a sample text."
text2 = "This is another text."
kl_div1= calculate_kl_divergence(text1, text2)
print("KL Divergence from text2 to text1:", kl_div1)




{'t': 0.14285714285714285, 'h': 0.047619047619047616, 'i': 0.09523809523809523, 's': 0.14285714285714285, ' ': 0.19047619047619047, 'a': 0.09523809523809523, 'm': 0.047619047619047616, 'p': 0.047619047619047616, 'l': 0.047619047619047616, 'e': 0.09523809523809523, 'x': 0.047619047619047616, 'o': 0, 'r': 0, 'n': 0}
KL Divergence from text2 to text1: 3.0544843900755074


In [ ]:
df

,section,title,abstract,url,Human story fetched,AI generated story,AI generated article with context
0,business,"Inflation slowed, good news as the Fed meets.",Federal Reserve officials are poised to announ...,https://www.nytimes.com/live/2024/06/12/busine...,U.S. Economy\nLive Updates: Inflation Cools Sl...,"## Inflation Cools Ahead of Fed Meeting, Offer...","## Inflation Cools, Offering Hope as Fed Holds..."
1,business,Buyers Snap Up Aging and Empty Office Building...,Bargain hunters are getting deals of up to 70 ...,https://www.nytimes.com/2024/06/12/business/di...,Supported by\nBuyers Snap Up Aging and Empty O...,## Second Life for Empty Towers: Investors See...,## Bargain Hunting Signals Deeper Trouble in C...
2,business,What to Watch as the Fed Meets,Federal Reserve officials are expected to leav...,https://www.nytimes.com/2024/06/12/business/ec...,U.S. Economy\nWhat to Watch as the Fed Meets\n...,## All Eyes on Powell: Fed Meeting to Determin...,## The Fed Holds Steady: A Waiting Game for Ra...
3,business,Losing a Bridge Upends Life in Baltimore (and ...,Congestion has increased on other routes in th...,https://www.nytimes.com/2024/06/12/business/ke...,BaltimoreBridge Collapse\nLosing a Bridge Upen...,## Losing a Bridge Upends Life in Baltimore (a...,## Losing a Bridge Upends Life in Baltimore (a...
4,business,Paramount’s Merger Talks With Skydance Fall Apart,There were several hitches in the last week as...,https://www.nytimes.com/2024/06/11/business/me...,Supported by\nParamount’s Merger Talks With Sk...,## Paramount and Skydance Merger Talks Collaps...,## Skydance and Paramount Merger Collapses in ...
...,...,...,...,...,...,...,...
207,climate,Nuclear Power Is Hard. A Climate-Minded Billio...,Work is starting in Wyoming coal country on a ...,https://www.nytimes.com/2024/06/11/climate/bil...,Climate Change\nClimate Change\nSupported by\n...,## Can a Climate-Conscious Billionaire Crack t...,## A Climate-Minded Billionaire's Gamble: Can ...
208,world,"After the hostage rescue, the U.N. says action...",Israel’s Parliament revived legislation that w...,https://www.nytimes.com/2024/06/11/world/middl...,Supported by\nKey Developments\nAfter the host...,## U.N. Raises War Crimes Concerns After Hosta...,## UN: Both Sides May Be Guilty of War Crimes ...
209,briefing,Tuesday Briefing,The U.N. passes a cease-fire resolution for Gaza.,https://www.nytimes.com/2024/06/11/briefing/is...,Supported by\nTuesday Briefing\nThe U.N. passe...,## Tuesday Briefing: [Insert most compelling/i...,## U.N. Demands Cease-fire in Gaza as Israel L...
210,world,Nigeria Confronts Its Worst Economic Crisis in...,People in Africa’s most populous nation are su...,https://www.nytimes.com/2024/06/11/world/afric...,Global Economy\nGlobal Economy\nSupported by\n...,"## Nigeria Grapples with Economic Crisis, Leav...",## Nigeria Grapples with Economic Crisis as Fo...


In [ ]:
data = df.copy()
# Assuming your dataframe is named 'data'
data["KL divergence - AI generated Zero Shot Story VS Human story"] = data.apply(lambda x: calculate_kl_divergence(x["AI generated story"], x["Human story fetched"] ), axis=1)
data["KL divergence - AI generated Zero Shot VS AI generated with context"] = data.apply(lambda x: calculate_kl_divergence(x["AI generated story"], x["AI generated article with context"]), axis=1)


{' ': 0.15544041450777202, 'm': 0.024964672633066416, 'i': 0.06971267074894018, 'd': 0.03297220913801225, 't': 0.07772020725388601, 'e': 0.09750353273669336, 'r': 0.05652378709373528, 'o': 0.06217616580310881, 'n': 0.0645313235986811, 'u': 0.01601507300989166, 'a': 0.07018370230805464, 'l': 0.04333490343853038, 'y': 0.011775788977861516, 's': 0.05134243994347621, 'w': 0.012246820536975978, 'v': 0.012246820536975978, 'g': 0.016486104569006125, 'b': 0.011304757418747056, 'k': 0.0037682524729156855, 'p': 0.019783325482807347, '\n': 0.006123410268487989, 'h': 0.03297220913801225, 'c': 0.03203014601978332, 'f': 0.01601507300989166, 'x': 0.001413094677343382, 'j': 0.001413094677343382}
{' ': 0.16430853491556366, 'a': 0.08215426745778183, 't': 0.06435417617526244, 'o': 0.06435417617526244, 'r': 0.05933363760839799, 'n': 0.06937471474212688, 'd': 0.03423094477407576, 'w': 0.02601551802829758, 'i': 0.05659516202647193, 'g': 0.020082154267457783, 'm': 0.016430853491556367, 'e': 0.100867183934276

<ipython-input-25-c14fc3f3e2a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["KL divergence - AI generated Zero Shot Story VS Human story"] = data.apply(lambda x: calculate_kl_divergence(x["AI generated story"], x["Human story fetched"] ), axis=1)


{' ': 0.1609858416360776, 'e': 0.10015731515469324, 'i': 0.06292606187729417, 'g': 0.019926586261143155, 'h': 0.03880440482433141, 't': 0.0713162034609334, 'd': 0.03093864708966964, 'a': 0.06449921342422653, 'o': 0.06345044572627163, 'z': 0.0026219192448872575, 'n': 0.05715783953854221, 's': 0.054535920293654956, 'j': 0.0026219192448872575, 'u': 0.023597273203985317, 'r': 0.06659674882013635, 'f': 0.025170424750917672, 'l': 0.028841111693759833, 'b': 0.0068169900367068695, 'c': 0.03408495018353435, 'v': 0.008390141583639224, 'm': 0.011536444677503933, 'w': 0.015731515469323543, 'k': 0.012585212375458836, '\n': 0.007865757734661772, 'y': 0.01101206082852648, 'p': 0.016255899318300997, 'x': 0.0015731515469323545, 'q': 0}
{' ': 0.16849615806805707, 'b': 0.01646542261251372, 'r': 0.04665203073545554, 'o': 0.066410537870472, 'n': 0.0570801317233809, 'y': 0.017014270032930844, 'j': 0.006586169045005488, 'a': 0.06695938529088913, 'm': 0.027991218441273325, 'e': 0.09604829857299671, 's': 0.061

<ipython-input-25-c14fc3f3e2a3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["KL divergence - AI generated Zero Shot VS AI generated with context"] = data.apply(lambda x: calculate_kl_divergence(x["AI generated story"], x["AI generated article with context"]), axis=1)


In [ ]:
data

,Category,NY Tweet,Tweet date (DD/MM/YYYY),NY Tweet URL,Human Generated Story,Human Generated Story URL,Prompt,Human story fetched,AI generated story,AI generated article with context,KL divergence - AI generated Zero Shot Story VS Human story,KL divergence - AI generated Zero Shot VS AI generated with context
0,Politics,"Voters in Maryland, West Virginia and Nebraska...",14/05/2024,https://twitter.com/nytimes/status/17904306427...,Will Democrats in Maryland back a former Capit...,https://www.nytimes.com/2024/05/14/us/politics...,"Generate article for the NY times tweet ""Voter...","\nWhat to Watch in Primaries in Maryland, Nebr...","## Midterm Momentum: All Eyes on Maryland, Wes...","## Maryland, Nebraska and West Virginia Primar...",0.011955,0.012118
1,Weather,The National Weather Service issues two types ...,14/05/2024,https://twitter.com/nytimes/status/17904269238...,The National Weather Service issues two types ...,https://www.nytimes.com/article/tornado-safety...,"Generate article for the NY times tweet ""The N...",Supported by\nWhat to Do When There’s a Tornad...,## A Tornado Warning Means Take Cover Now: He...,## A Tornado Warning Demands Action: Here's Wh...,0.035915,0.013382
2,History,Civil War-era artifacts like weapons from the ...,14/05/2024,https://twitter.com/nytimes/status/17904230902...,"Weapons from the battlefield, books inscribed ...",https://www.nytimes.com/2024/05/14/us/civil-wa...,"Generate article for the NY times tweet ""Civil...",Supported by\nSherman Turned the Tide of the C...,## Sherman's Shadow: Civil War Relics Head to ...,## Sherman Turned the Tide of the Civil War. H...,0.021826,0.013601
3,Crime,At least eight people were killed and dozens w...,14/05/2024,https://twitter.com/nytimes/status/17904206341...,At least eight people were killed and dozens o...,https://www.nytimes.com/2024/05/14/us/florida-...,"Generate article for the NY times tweet ""At le...",Supported by\nPickup Driver Arrested After Cra...,"## Eight Dead, Dozens Injured in Florida Bus C...",## Eight Migrant Farm Workers Killed in Florid...,0.037401,0.016584
4,Sports,"From \n@TheAthletic\n: Bronny James, the son o...",14/05/2024,https://twitter.com/nytimes/status/17904168156...,Bronny James is expected to stay in the NBA Dr...,https://www.nytimes.com/athletic/5492505/2024/...,"Generate article for the NY times tweet ""From ...",NBA\nWhat we learned — and didn’t — about Bron...,## Bronny James Shines on Day One of NBA Draft...,## Bronny James at the NBA Draft Combine: What...,0.013888,0.021551


In [ ]:
df.to_csv("Gemini experiments_212.csv")

## Fetching Top article URLs, Title and Abstract from different sections



In [ ]:
from pynytimes import NYTAPI

api_key = "zJK0Mfsv3SUGBsIMlm9h0rXS6NEtDAAu"

nyt = NYTAPI(api_key, parse_dates=True)

In [ ]:
# All possible categories you can use, You can take a subset of it

categories = [
    "arts", "automobiles", "books", "business", "fashion", "food", "health",
    "home", "insider", "magazine", "movies", "national", "nyregion",
    "obituaries", "opinion", "politics", "realestate", "science", "sports",
    "sundayreview", "technology", "theater", "tmagazine", "travel", "upshot",
    "world"
]


### Experimenting to see how to get the desired output

In [ ]:

top_stories = nyt.top_stories()

# Get all the top stories from a specific category
top_science_stories = nyt.top_stories()
len(top_science_stories)

27

In [ ]:
top_science_stories[0]

{'section': 'world',
 'subsection': 'middleeast',
 'title': 'Israel’s Supreme Court Rules the Military Must Draft Ultra-Orthodox Jews',
 'abstract': 'The court ruled there was no basis to exempt the ultra-Orthodox from service, a decision that threatened to split Prime Minister Benjamin Netanyahu’s government amid the war in Gaza.',
 'url': 'https://www.nytimes.com/2024/06/25/world/middleeast/israel-military-ultra-orthodox-jews-supreme-court.html',
 'uri': 'nyt://article/df8f7d44-67de-539e-84ac-ab9912e42cda',
 'byline': 'By Aaron Boxerman',
 'item_type': 'Article',
 'updated_date': datetime.datetime(2024, 6, 25, 7, 42, 22, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),
 'created_date': datetime.datetime(2024, 6, 25, 4, 25, 58, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),
 'published_date': datetime.datetime(2024, 6, 25, 4, 25, 58, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),
 'material_type_facet': '',
 'kicker

In [ ]:
top_science_stories[0]["url"]

'https://www.nytimes.com/2024/06/12/business/distressed-office-buildings-buyers.html'

In [ ]:
top_science_stories[0]["section"]

'business'

In [ ]:
top_science_stories[0]["title"]

'Buyers Snap Up Aging and Empty Office Buildings for Deep Discounts'

In [ ]:
top_science_stories[0]["abstract"]

'Bargain hunters are getting deals of up to 70 percent, a sign of the pain in the commercial property market that could lead to large losses for banks and investors in real-estate-backed loans.'

In [ ]:
import pandas as pd

def fetch_article_data(categories):
  """Fetches article URLs, titles, and abstracts from The New York Times for specified categories and stores them in a pandas DataFrame.

  Args:
    categories (list): A list of The New York Times section names to fetch articles from.

  Returns:
    pandas.DataFrame: A DataFrame containing the collected article data (if successful).
  """

  # Create an empty DataFrame to store the article data
  article_data = pd.DataFrame(columns=["section", "title", "abstract","url"])

  # Create a New York Times API client (replace with your API key)
  api_key = "zJK0Mfsv3SUGBsIMlm9h0rXS6NEtDAAu"

  nyt = NYTAPI(api_key, parse_dates=True)

  for category in categories:
    # Fetch top stories from the specified category
    top_stories = nyt.top_stories(section=category)

    for story in top_stories:
      # Extract and store URL, title, and abstract in the DataFrame
      article_data = article_data._append({
          "section": story["section"],
          "title": story["title"],
          "abstract": story["abstract"],
          "url": story["url"]
      }, ignore_index=True)


  return article_data

# Example usage (replace 'YOUR_NEW_YORK_TIMES_API_KEY' with your actual API key)
categories = [
    "business", "food", "health",
    "politics", "science",
    "technology", "world"
]

df = fetch_article_data(categories)


# We are going to use this data for implementing our previous experiments

In [ ]:
df.head(5)

,section,title,abstract,url
0,business,"Inflation slowed, good news as the Fed meets.",Federal Reserve officials are poised to announ...,https://www.nytimes.com/live/2024/06/12/busine...
1,business,Buyers Snap Up Aging and Empty Office Building...,Bargain hunters are getting deals of up to 70 ...,https://www.nytimes.com/2024/06/12/business/di...
2,business,What to Watch as the Fed Meets,Federal Reserve officials are expected to leav...,https://www.nytimes.com/2024/06/12/business/ec...
3,business,Losing a Bridge Upends Life in Baltimore (and ...,Congestion has increased on other routes in th...,https://www.nytimes.com/2024/06/12/business/ke...
4,business,Paramount’s Merger Talks With Skydance Fall Apart,There were several hitches in the last week as...,https://www.nytimes.com/2024/06/11/business/me...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212 entries, 0 to 214
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   section              212 non-null    object
 1   title                212 non-null    object
 2   abstract             212 non-null    object
 3   url                  212 non-null    object
 4   Human story fetched  212 non-null    object
dtypes: object(5)
memory usage: 9.9+ KB


In [ ]:
df.to_csv("NYT_prompts.csv")

In [ ]:
df = df.dropna()

In [ ]:
# Fetch article text sequentially for each URL and store it in a new attribute 'Human story fetched'
df['Human story fetched'] = df['url'].apply(fetch_article_text_sequential)


In [ ]:
data["Human Generated Story URL"]

0    https://www.nytimes.com/2024/05/14/us/politics...
1    https://www.nytimes.com/article/tornado-safety...
2    https://www.nytimes.com/2024/05/14/us/civil-wa...
3    https://www.nytimes.com/2024/05/14/us/florida-...
4    https://www.nytimes.com/athletic/5492505/2024/...
Name: Human Generated Story URL, dtype: object

In [ ]:
def process_and_analyze_articles(NYT_data, text_column, url_column):
  """
  Processes and analyzes articles in a DataFrame.

  Args:
      df (pandas.DataFrame): The DataFrame containing article data.
      text_column (str): The column name containing the text used for zero-shot generation (e.g., 'NY Tweet').
      url_column (str): The column name containing the URLs of human-generated stories.
      fetch_article_text_sequential (function): A function to fetch article text sequentially for each URL.
      ai_generation_without_context (function): A function to generate an AI-generated article with zero-shot learning.
      ai_generation_with_context (function): A function to generate an AI-generated article with context.
      calculate_kl_divergence (function): A function to calculate KL divergence between two strings.

  Returns:
      pandas.DataFrame: The modified DataFrame with additional columns for fetched human stories,
                        AI-generated articles (zero-shot and with context), and KL divergences.
  """

  # Fetch human stories sequentially
  NYT_data['Human story fetched'] = NYT_data[url_column].apply(fetch_article_text_sequential)

  display(NYT_data)

  # Generate AI-generated articles
  NYT_data['AI generated story'] = NYT_data[text_column].apply(ai_generation_without_context)
  NYT_data['AI generated article with context'] = NYT_data.apply(lambda row: ai_generation_with_context(row[text_column], row['Human story fetched']), axis=1)

  display(df)

  # Calculate KL divergences
  NYT_data["KL divergence - AI generated Zero Shot Story VS Human story"] = NYT_data.apply(lambda x: calculate_kl_divergence(x["AI generated story"], x["Human story fetched"]), axis=1)
  NYT_data["KL divergence - AI generated Zero Shot VS AI generated with context"] = NYT_data.apply(lambda x: calculate_kl_divergence(x["AI generated story"], x["AI generated article with context"]), axis=1)

  display(NYT_data)

  return NYT_data



In [ ]:
# Example usage (assuming you have the necessary functions)
processed_data = process_and_analyze_articles(df.copy(), 'abstract', 'url')

# Display the relevant columns
print(processed_data[["Human story fetched", "AI generated story", "AI generated article with context",
                     "KL divergence - AI generated Zero Shot Story VS Human story",
                     "KL divergence - AI generated Zero Shot VS AI generated with context"]])


,url,title,abstract,Human story fetched
0,https://www.nytimes.com/2024/06/10/technology/...,Welcome to the Era of the A.I. Smartphone,Apple and Google are getting up close and pers...,ArtificialIntelligence\nArtificialIntelligence...
1,https://www.nytimes.com/2024/06/10/technology/...,Apple Jumps Into A.I. Fray With Apple Intellig...,"The iPhone maker, which has been slow to embra...",ArtificialIntelligence\nArtificialIntelligence...
2,https://www.nytimes.com/2024/06/10/business/bo...,A Big Decision for Boeing’s Next C.E.O.: Is It...,Some analysts say building a new plane soon wo...,A Big Decision for Boeing’s Next C.E.O.: Is It...
3,https://www.nytimes.com/2024/06/09/technology/...,Is Your Driving Being Secretly Scored?,"The insurance industry, hungry for insights in...",Supported by\nIs Your Driving Being Secretly S...
4,https://www.nytimes.com/2024/06/08/business/rh...,The Napoleon of Your Living Room,"Gary Friedman, who runs RH (formerly Restorati...",The Napoleon of Your Living Room\nGary Friedma...
...,...,...,...,...
210,https://www.nytimes.com/2024/06/09/world/asia/...,Monday Briefing: Macron Calls New French Legis...,"Also, Benny Gantz leaves Israel’s government.",None
211,https://www.nytimes.com/2024/06/09/world/ameri...,Mexico’s Leftists Won Big. Investors Are Worried.,The peso had its worst week since the pandemic...,None
212,https://www.nytimes.com/2024/06/09/world/europ...,Flemish Nationalists Thwart Ascent of Secessio...,The New Flemish Alliance’s victory will bring ...,None
213,https://www.nytimes.com/2024/06/09/world/europ...,"In E.U. Elections, the Center Holds, but the F...",Voters in the European Union delivered strong ...,None


ValueError: Your location is not supported by google-generativeai at the moment. Try to use ChatVertexAI LLM from langchain_google_vertexai.

In [ ]:
NYT_data = df.copy()

In [ ]:
for index, row in df.head(5).iterrows():
    # print(row)
    url_column = row["url"]
    text_column = row["abstract"]


    # Fetch human stories sequentially
    NYT_data['Human story fetched'] = NYT_data[url_column].apply(fetch_article_text_sequential)

    display(NYT_data)

    # Generate AI-generated articles
    NYT_data['AI generated story'] = NYT_data[text_column].apply(ai_generation_without_context)
    NYT_data['AI generated article with context'] = NYT_data.apply(lambda row: ai_generation_with_context(row[text_column], row['Human story fetched']), axis=1)

    display(df)

    # Calculate KL divergences
    NYT_data["KL divergence - AI generated Zero Shot Story VS Human story"] = NYT_data.apply(lambda x: calculate_kl_divergence(x["AI generated story"], x["Human story fetched"]), axis=1)
    NYT_data["KL divergence - AI generated Zero Shot VS AI generated with context"] = NYT_data.apply(lambda x: calculate_kl_divergence(x["AI generated story"], x["AI generated article with context"]), axis=1)

    display(NYT_data)

KeyError: 'https://www.nytimes.com/2024/06/10/technology/personaltech/apple-google-ai.html'

In [ ]:
fetch_article_text_sequential(NYT_data["url"].head(1))

InvalidSchema: No connection adapters were found for '0    https://www.nytimes.com/2024/06/10/technology/...\nName: url, dtype: object'

In [ ]:
NYT_data['Human story fetched'] = NYT_data['url'].apply(fetch_article_text_sequential)


In [ ]:
NYT_data

,url,title,abstract,Human story fetched
0,https://www.nytimes.com/2024/06/10/technology/...,Welcome to the Era of the A.I. Smartphone,Apple and Google are getting up close and pers...,ArtificialIntelligence\nArtificialIntelligence...
1,https://www.nytimes.com/2024/06/10/technology/...,Apple Jumps Into A.I. Fray With Apple Intellig...,"The iPhone maker, which has been slow to embra...",ArtificialIntelligence\nArtificialIntelligence...
2,https://www.nytimes.com/2024/06/10/business/bo...,A Big Decision for Boeing’s Next C.E.O.: Is It...,Some analysts say building a new plane soon wo...,A Big Decision for Boeing’s Next C.E.O.: Is It...
3,https://www.nytimes.com/2024/06/09/technology/...,Is Your Driving Being Secretly Scored?,"The insurance industry, hungry for insights in...",Supported by\nIs Your Driving Being Secretly S...
4,https://www.nytimes.com/2024/06/08/business/rh...,The Napoleon of Your Living Room,"Gary Friedman, who runs RH (formerly Restorati...",The Napoleon of Your Living Room\nGary Friedma...
...,...,...,...,...
210,https://www.nytimes.com/2024/06/09/world/asia/...,Monday Briefing: Macron Calls New French Legis...,"Also, Benny Gantz leaves Israel’s government.",Newsletter\nMorning Briefing: Asia Pacific Edi...
211,https://www.nytimes.com/2024/06/09/world/ameri...,Mexico’s Leftists Won Big. Investors Are Worried.,The peso had its worst week since the pandemic...,Mexico’s PresidentialElection\nMexico’s Presid...
212,https://www.nytimes.com/2024/06/09/world/europ...,Flemish Nationalists Thwart Ascent of Secessio...,The New Flemish Alliance’s victory will bring ...,InternationalElections 2024\nInternationalElec...
213,https://www.nytimes.com/2024/06/09/world/europ...,"In E.U. Elections, the Center Holds, but the F...",Voters in the European Union delivered strong ...,E.U. ParliamentElections\nE.U. ParliamentElect...


In [ ]:
# Check for missing URLs
missing_urls = NYT_data['url'].notna()


In [ ]:
missing_urls.sum()

215

In [ ]:
def fix_url(url):
  if not url.startswith('https'):
    # url = 'https://' + url
    print(url,"hi")
  return url

NYT_data['url'].apply(fix_url)


 hi
null hi
 hi


0      https://www.nytimes.com/2024/06/10/technology/...
1      https://www.nytimes.com/2024/06/10/technology/...
2      https://www.nytimes.com/2024/06/10/business/bo...
3      https://www.nytimes.com/2024/06/09/technology/...
4      https://www.nytimes.com/2024/06/08/business/rh...
                             ...                        
210    https://www.nytimes.com/2024/06/09/world/europ...
211    https://www.nytimes.com/2024/06/09/world/middl...
212    https://www.nytimes.com/2024/06/09/world/middl...
213    https://www.nytimes.com/2024/06/09/world/middl...
214    https://www.nytimes.com/2024/06/09/world/middl...
Name: url, Length: 215, dtype: object